###  Cell Phone reviews (78,930 reviews)
[Вот тут брала корпус](http://snap.stanford.edu/data/amazon/Cell_Phones_&_Accessories.txt.gz)

In [2]:
!pip install simplejson

In [3]:
import gzip
import simplejson

def parse(filename):
  f = gzip.open(filename, 'rt')
  entry = {}
  for l in f:
    l = l.strip()
    colonPos = l.find(':')
    if colonPos == -1:
      yield entry
      entry = {}
      continue
    eName = l[:colonPos]
    rest = l[colonPos+2:]
    entry[eName] = rest
  yield entry


data = []
for e in parse("/content/drive/MyDrive/PROGA/автобрея/HW2/Cell_Phones_&_Accessories.txt.gz"):
    data.append(simplejson.dumps(e))


In [4]:
data = [eval(x) for x in data]
type(data[0])

dict

In [5]:
data[0]

{'product/price': 'unknown',
 'product/productId': 'B000JVER7W',
 'product/title': 'Mobile Action MA730 Handset Manager - Bluetooth Data Suite',
 'review/helpfulness': '0/0',
 'review/profileName': 'A. Igoe',
 'review/score': '1.0',
 'review/summary': "Don't buy!",
 'review/text': "First of all, the company took my money and sent me an email telling me the product was shipped. A week and a half later I received another email telling me that they are sorry, but they don't actually have any of these items, and if I received an email telling me it has shipped, it was a mistake.When I finally got my money back, I went through another company to buy the product and it won't work with my phone, even though it depicts that it will. I have sent numerous emails to the company - I can't actually find a phone number on their website - and I still have not gotten any kind of response. What kind of customer service is that? No one will help me with this problem. My advice - don't waste your money!"

### Способы  найти упоминания товаров в отзывах:

1. **Первый способ**

 Составим  биграммы на основе наименований товаров + воспользуемся stanza для нахождения именованных сущностей. Для второго посмотрим, какие типы он выделяет и возьмем только некоторые, например: 
- entity: Motorola	type: ORG - как раз то, что нам нужно
- entity: 12 hours	type: TIME - такое нам, очевидно не нужно

UPD: как оказалось, stanza хорошо находит сущности в тайтлах продуктов, но не в текстах отзывов. Поэтому вместе с биграммами будем использовать эмбединги spicy.
Для фильтрации сущностей по результатам анализа выборки взяли следующие лейблы:
- FAC =	Buildings, airports, highways, bridges, etc. (также выделяет некоторые электронные приборы: a Cell Tower -> FAC)
- ORG = Companies, agencies, institutions, etc. (возвращает нам названия фирм - Motorola)
- PRODUCT =	Objects, vehicles, foods, etc.
- WORK_OF_ART =	Titles of books, songs, etc. (находит некоторые наименования товаров: Jabra EarGels -> WORK_OF_ART)
- LOC = Non-GPE locations, mountain ranges, bodies of water.(Находит названия товаров, так как часто там используются такие слова : Arctic Cooling -> LOC)
- PERSON = People, including fictional. (попробовала найти такие лейблы, получилось, что это очень даже нам подходит, тк людей в отзывах не упоминают, но он возыращает разные названия: Sony Ericsson -> PERSON)

Таким образом, у нас получается словарь, созданный объединением двух способов: поиска биграмм, обученныеx на заголовках + тфидф и поиска именнованных сущностей spicy(определенных лейблов) 

**Достоинства и недостатки:** В отличии от других методов, кроме текстов самих отзывов нужны еще заголовки для создания биграмм (а сами отзывы нужны для поиска именнованных сущностей). Главная проблема в том, что много мусора, несмотря на то, что мы обучали исключительно на заголовках = наименованиях товаров

2. **Второй способ:**  Подход на основе правил. Например, можно написать правила с помощью natasha/yargy или другой библиотеки.

**Достоинства и недостатки:** Плюс в том, что вывод будет более-менее чистый, без мусора, благодаря четким критериям. Но, к сожалению, составитель правил может не учесть все возможные варианты или сделать их неоптимально, что очень вероятно, если писать правила вручную. С другой стороны, можно легко ограничить выходной результат строгими критериями (если этого требет задача) + здесь можно, например, задействовать правила-шаблоны, опирающиеся на частеречные теги + если нужна определенная последовательность тегов или даже можно учесть какие-либо грамматические категории. То есть такая задача сводится к поиску шаблона - последовательности частеречных тегов и, если нужно, грамматической информации.

3. **Третий способ** Подход на основе словаря.

**Достоинства и недостатки:** Главный плюс, что словарь можно обновлять. Более того, как и в предыдущем случае, если составлять словарь ручками, то результат получится хороший, так как не будет мусора в найденных объектах. Но проблемы остаются те же, что и с rule-based подходом. Хотя, мне кажется, используя словарь, будет все-таки меньше мусора, так как правила могут захватывать лишнюю информацию, а если у нас составленный вручную словарь, он никак ничего лишнего не найдет. Кажется, что можно безболезненно увеличить словарь, не сильно добавив мусора, с помощью поиска синонимичных вещей (например, как у меня дальше в примере, только взять нужно, конечно, другую модель)



### Реализуем первый способ поиска упоминания товаров в отхывах

In [ ]:
!pip install torch==1.7.0
!pip install stanza
import gensim
import pandas as pd
from gensim.models import Phrases
import stanza
stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')  
nltk.download('punkt')
from string import punctuation
import spacy
import en_core_web_sm
model_sp = en_core_web_sm.load()
stopwords = stopwords.words('english')
punctuation = punctuation + '„“)('
stopwords.append('for')
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
from tqdm import tqdm

In [7]:
reviews = []
titles = []
for d in data:
    try:
      reviews.append(d['review/text'])
      titles.append(d['product/title'])

    except KeyError:
        pass
len(data), len(reviews), len(titles)

(78931, 78930, 78930)

In [8]:
df = pd.DataFrame({'titles':titles, 'reviews':reviews})

In [9]:
# токенизируем и лемматизируем тексты + убирам пунктуацию и стоп слова
# проблема в том, что лемматизатор возвращает все в нижнем регистре,
# попробуем это исправить
def normalize(text):
  # Take input text and return list of lemmas
  lemmas = []
  for t in tokenizer.tokenize(text):
    if t[0].isupper()== True:
      lemmas.append(m.parse(t)[0].normal_form.title())
    else:
      lemmas.append(m.parse(t)[0].normal_form)
  lemmas = [word for word in lemmas if word not in stopwords]
  lemmas = [(sentence.translate(str.maketrans('', '', punctuation))) for sentence in lemmas]
    
  return lemmas

normalize_reviews = [normalize(text) for text in tqdm(reviews)]
df['normalize_reviews'] = normalize_reviews

100%|██████████| 78930/78930 [03:23<00:00, 388.51it/s]


In [10]:
df.shape

(78930, 3)

In [11]:
# Найдем нужные сущности с помощью spicy
def spicy_find(reviews: list):
  labels = ['PERSON','LOC','FAC', 'ORG','PRODUCT', 'WORK_OF_ART']
  for doc in reviews:
    for ent in model_sp(doc).ents:
      if ent.label_ in labels:
        print(ent.text, '->', ent.label_)
spicy_find(reviews[:10])

S&H -> ORG
Sony -> ORG
Amazon -> ORG
the Phone Monitor Options -> ORG
C:\Program -> ORG
Options -> ORG
COM -> ORG
COM -> ORG
USB -> ORG
Sony Ericsson K700i -> ORG
the USB Data Cable -> ORG
MyNetDeals -> PERSON
Sony Ericsson -> PERSON
rude & -> ORG
wallpaper & ringtones -> ORG
Sony -> ORG
Sony -> ORG
Sony Data -> ORG
Sony -> ORG
Sony -> ORG
3rd party -> ORG
Sony -> ORG
Sony Ericsson -> ORG
Sony -> ORG
Sony -> ORG
Sony Ericsson z500a & watch -> ORG
Electro Island -> FAC


In [12]:
# датасет слишком большой, так что возьмем только часть
# заодно возьмем только уникальные заголовки
small_df = pd.DataFrame(columns = df.columns)
product_names = []
for i, el in enumerate(df.titles):
  if el not in product_names:
    product_names.append(el)
    small_df = small_df.append(df.loc[i])

In [13]:
small_df.shape

(7150, 3)

In [14]:
small_df.head()

,titles,reviews,normalize_reviews
0,Mobile Action MA730 Handset Manager - Bluetoot...,"First of all, the company took my money and se...","[First, company, took, money, sent, email, tel..."
3,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",The price was right for this cable ($11.95+$4....,"[The, price, right, cable, 11, 95, 4, 94, S, H..."
12,Motorola HT820 Stereo Bluetooth Headset with m...,"A great device, I use it with a NOKIA 9300 it ...","[A, great, device, I, use, Nokia, 9300, great,..."
17,,"I would like to buy two or more shells, do you...","[I, would, like, buy, two, shells, offer, diff..."
18,Belkin Executive Leather Case handheld carryin...,This leather cover provides a professional fin...,"[This, leather, cover, provides, professional,..."


In [15]:
labels = ['PERSON','LOC','FAC', 'ORG','PRODUCT', 'WORK_OF_ART']
entities = []
for doc in small_df.reviews:
  ents = []
  for ent in model_sp(doc).ents:
    if ent.label_ in labels:
      # print(ent.text)
      ents.append(ent.text)
  entities.append(ents)
small_df['entities_spicy'] = entities

In [16]:
small_df.head(15)

,titles,reviews,normalize_reviews,entities_spicy
0,Mobile Action MA730 Handset Manager - Bluetoot...,"First of all, the company took my money and se...","[First, company, took, money, sent, email, tel...",[]
3,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",The price was right for this cable ($11.95+$4....,"[The, price, right, cable, 11, 95, 4, 94, S, H...","[S&H, Sony, Amazon, the Phone Monitor Options,..."
12,Motorola HT820 Stereo Bluetooth Headset with m...,"A great device, I use it with a NOKIA 9300 it ...","[A, great, device, I, use, Nokia, 9300, great,...",[NOKIA]
17,,"I would like to buy two or more shells, do you...","[I, would, like, buy, two, shells, offer, diff...",[]
18,Belkin Executive Leather Case handheld carryin...,This leather cover provides a professional fin...,"[This, leather, cover, provides, professional,...",[]
19,Samsung T809 M620 i607 U420 A303 A717 Car Char...,I thought this item was great for the money. I...,"[I, thought, item, great, money, If, would, bo...",[Verizon]
21,For Motorola V9m/ V3/ V3c/ V3i/ V3m/ V3t/ V3v ...,Great case... cingular should get their cases ...,"[Great, case, cingular, get, cases, supplier, ...",[]
22,NEW UNLOCKED SAMSUNG X830 PINK MP3 CAMERA GSM ...,This item came earlier than expected! It is ev...,"[This, item, came, earlier, expected, It, ever...",[]
23,Ericsson Dual Slot Desktop Charger for Ericsso...,I purchased this charger along with the &quot;...,"[I, purchased, charger, along, quot, Ericsson,...","[the &quot;Ericsson, LX588, Charger, Ericsson]"
24,Audiovox Leather Carrying Case for Audiovox Ph...,The case I had didn't fit properly - the windo...,"[The, case, I, fit, properly, window, obstruct...",[]


In [17]:
# Попробуем найти биграммы заголовков 
product_names = titles

In [18]:
sentences = [x.split() for x in product_names if x not in stopwords]
unigram_sentences = []
for text in sentences:
    new_text = []
    for word in text:
        if word not in punctuation:
            new_text.append(word)
    unigram_sentences.append(new_text)

In [19]:
# to create the bigrams
bigram_model = Phrases(unigram_sentences)

bigram_sentences = []
# apply the trained model to a sentence
for unigram_sentence in unigram_sentences:
    bigram_sentence = bigram_model[unigram_sentence]
    bigram_sentences.append(bigram_sentence)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
text1_tfidf = vectorizer.transform([" ".join(bigram_sentences[0])])
vocab = vectorizer.get_feature_names()
tfidf = vectorizer.idf_
top = list(sorted(zip(vocab, text1_tfidf.toarray()[0]), key=lambda x: x[1], reverse = True))
for x,y in top:
  if y > 0.4:
    print(x,' -> ', y)

In [50]:
# сделаем словарь из уникальных биграмм
bigrams_titles = []
for x in bigram_sentences:
  for y in x:
    if '_' in y:
      bigrams_titles.append(y)

SyntaxError: ignored

In [52]:
bigrams_titles_uniq = list(set(bigrams_titles))
print(bigrams_titles_uniq[:10])
len(bigrams_titles_uniq)

['(Black color', 'BlueAnt X5', 'T-MOBILE GSM', 'T809, D807,', 'Pro Duo', 'L7e L7i', 'SNN5760A BATTERY', 'C1300 CE500', 'CTA Digital', 'C1500 C2000']


4326

In [55]:
texts = list(small_df.normalize_reviews)
for text in texts[:50]:
  for bi in bigrams_titles_uniq:
    if bi in ' '.join(text):
      print(bi)

Hands Free
hands free
cell phone
hands free
data cable
cell phone
phone cover
cell phone
cell phone
cell phone
hands free
cell phone


In [57]:
vocavb_bi = []
for text in texts:
  each = []
  for bi in bigrams_titles_uniq:
    if bi in ' '.join(text):
      each.append(bi)
  vocavb_bi.append(each)

In [20]:
bigrams_reviews = []
texts = list(small_df.normalize_reviews)
for text in texts:
  bigrams_reviews.append(bigram_model[text])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(" ".join(t) for t in bigram_sentences)
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [59]:
bigram_idf = []
for text in bigrams_reviews:
  tf = []
  text1_tfidf = vectorizer.transform([" ".join(text)])
  vocab = vectorizer.get_feature_names()
  tfidf = vectorizer.idf_
  top = list(sorted(zip(vocab, text1_tfidf.toarray()[0]), key=lambda x: x[1], reverse = True))
  for x,y in top:
    if y > 0.4:
      tf.append(x)
  bigram_idf.append(tf)

In [60]:
# если честно, выглядит такое себе, не понимаю, в чем проблема
bigram_idf[:15]

[['email', 'sent', 'product'],
 [],
 ['great', 'hear'],
 ['two', 'pay', 'buy', 'choose'],
 ['computer'],
 ['item', 'great'],
 ['cingular', 'get', 'great'],
 [],
 [],
 ['shift', 'one'],
 [],
 ['ear', 'jawbone'],
 ['it', 'good'],
 ['short'],
 ['good', 'product', 'buy']]

In [61]:
len(vocavb_bi), len(bigram_idf), len(entities)

(7150, 7150, 7150)

In [63]:
final_entities = []
for i in range(len(vocavb_bi)): 
  final = [*vocavb_bi[i],*bigram_idf[i],*entities[i]]
  final = list(set(final))
  final_entities.append(final)

In [64]:
small_df['final_entities'] = final_entities
small_df.head(15)

,titles,reviews,normalize_reviews,entities_spicy,final_entities
0,Mobile Action MA730 Handset Manager - Bluetoot...,"First of all, the company took my money and se...","[First, company, took, money, sent, email, tel...",[],"[product, email, sent]"
3,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",The price was right for this cable ($11.95+$4....,"[The, price, right, cable, 11, 95, 4, 94, S, H...","[S&H, Sony, Amazon, the Phone Monitor Options,...","[COM, C:\Program, Amazon, S&H, the Phone Monit..."
12,Motorola HT820 Stereo Bluetooth Headset with m...,"A great device, I use it with a NOKIA 9300 it ...","[A, great, device, I, use, Nokia, 9300, great,...",[NOKIA],"[hear, great, NOKIA]"
17,,"I would like to buy two or more shells, do you...","[I, would, like, buy, two, shells, offer, diff...",[],"[choose, buy, two, pay]"
18,Belkin Executive Leather Case handheld carryin...,This leather cover provides a professional fin...,"[This, leather, cover, provides, professional,...",[],[computer]
19,Samsung T809 M620 i607 U420 A303 A717 Car Char...,I thought this item was great for the money. I...,"[I, thought, item, great, money, If, would, bo...",[Verizon],"[great, item, Verizon]"
21,For Motorola V9m/ V3/ V3c/ V3i/ V3m/ V3t/ V3v ...,Great case... cingular should get their cases ...,"[Great, case, cingular, get, cases, supplier, ...",[],"[get, great, cingular]"
22,NEW UNLOCKED SAMSUNG X830 PINK MP3 CAMERA GSM ...,This item came earlier than expected! It is ev...,"[This, item, came, earlier, expected, It, ever...",[],[]
23,Ericsson Dual Slot Desktop Charger for Ericsso...,I purchased this charger along with the &quot;...,"[I, purchased, charger, along, quot, Ericsson,...","[the &quot;Ericsson, LX588, Charger, Ericsson]","[Ericsson, LX588, the &quot;Ericsson, Charger]"
24,Audiovox Leather Carrying Case for Audiovox Ph...,The case I had didn't fit properly - the windo...,"[The, case, I, fit, properly, window, obstruct...",[],"[one, shift]"


### Способ объединить синонимичные упоминания

In [61]:
# попробуем найти способ объединить синонимичные упоминания
from gensim.models import phrases, word2vec
model = word2vec.Word2Vec(bigram_sentences, size=50, min_count=3, iter=20)
model.wv.most_similar('Leather_Case')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Belt_Clip', 0.9606925845146179),
 ('Holster', 0.9494993090629578),
 ('Clip', 0.9392268061637878),
 ('Platinum_Skin', 0.9307593107223511),
 ('Ratcheting', 0.9305804967880249),
 ('Leather_Wrapped', 0.9280011057853699),
 ('Leather_Pouch', 0.9260797500610352),
 ('Long', 0.9246401786804199),
 ('Plastic_Swivel', 0.920512855052948),
 ('Closure', 0.919310986995697)]

## Неудачная попытка найти NER c помощью stanza

In [ ]:
doc = nlp(' '.join(sentences[13]))
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')

entity: Nokia	type: ORG


In [ ]:
# to create the bigrams
bigram_model = Phrases(unigram_sentences)

bigram_sentences = []
# apply the trained model to a sentence
for unigram_sentence in unigram_sentences:
    bigram_sentence = bigram_model[unigram_sentence]
    bigram_sentences.append(bigram_sentence)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
bigram_sentences[4] 

['Belkin',
 'Executive',
 'Leather_Case',
 'handheld',
 'carrying',
 'case',
 'F8D0402']

In [ ]:
# попробуем найти способ объединить синонимичные упоминания
from gensim.models import phrases, word2vec
model = word2vec.Word2Vec(bigram_sentences, size=50, min_count=3, iter=20)
model.wv.most_similar('Leather_Case')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Belt_Clip', 0.9567947387695312),
 ('Ratcheting', 0.9527748823165894),
 ('Leather_Wrapped', 0.9432302117347717),
 ('Leather_Pouch', 0.9420849084854126),
 ('Long', 0.9393155574798584),
 ('Holster', 0.9325529336929321),
 ('Universal_Horizontal', 0.929357647895813),
 ('Plastic_Swivel', 0.9121782183647156),
 ('Swivel_Belt', 0.9098306894302368),
 ('Krusell', 0.90790855884552)]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5)

tfidf_matrix = vectorizer.fit_transform(" ".join(t) for t in bigram_sentences)
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [27]:
print(" ".join(bigram_sentences[0]))
text1_tfidf = vectorizer.transform([" ".join(bigram_sentences[0])])
vocab = vectorizer.get_feature_names()
tfidf = vectorizer.idf_
top = list(sorted(zip(vocab, text1_tfidf.toarray()[0]), key=lambda x: x[1], reverse = True))
for x,y in top:
  if y > 0.4:
    print(x,' -> ', y)

Mobile_Action MA730 Handset_Manager Bluetooth Data Suite
ma730  ->  0.5559198315357913
handset_manager  ->  0.4565840584981748
suite  ->  0.45138820573691657


In [ ]:
doc = nlp(' '.join(sentences[13]))
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')

entity: Nokia	type: ORG


In [ ]:
texts = []
for d in data:
    try:
      texts.append(d['review/text'])
    except KeyError:
        pass

In [ ]:
from tqdm import tqdm
def find_products(text):
  doc = nlp(' '.join(text))
  result = [f'entity: {ent.text}, type: {ent.type}' for ent in doc.ents]
  return result

# возьмем пока только 1000, иначе долго работает
results = []
for text in tqdm(texts[:1000]):
  find = find_products(text)
  if find != None:
    results.append(find)

100%|██████████| 1000/1000 [23:10<00:00,  1.39s/it]


In [ ]:
# к сожалению, нужных сущностей, как в примере, мы не выделили
# Я не понимаю, почему в заголовках он много всего находит,
# А в текстах ничего нужного
results = [x for x in results if x != []]
results 

[['entity: 6, type: CARDINAL'],
 ['entity: 9, type: MONEY'],
 ['entity: 1, type: CARDINAL', 'entity: 2, type: CARDINAL'],
 ['entity: 0, type: CARDINAL'],
 ['entity: 4, type: MONEY'],
 ['entity: 3, type: CARDINAL', 'entity: 4, type: CARDINAL'],
 ['entity: 2, type: CARDINAL'],
 ['entity: 3, type: CARDINAL'],
 ['entity: 3, type: CARDINAL'],
 ['entity: 2, type: CARDINAL'],
 ['entity: $ 2 0, type: MONEY'],
 ['entity: 1, type: CARDINAL',
  'entity: 2, type: CARDINAL',
  'entity: 4, type: CARDINAL',
  'entity: 1, type: CARDINAL',
  'entity: 4, type: CARDINAL'],
 ['entity: 9, type: CARDINAL'],
 ['entity: 5   s, type: QUANTITY'],
 ['entity: 5, type: CARDINAL'],
 ['entity: 9 2, type: CARDINAL',
  'entity: 1 2, type: CARDINAL',
  'entity: 1, type: CARDINAL'],
 ['entity: 4, type: CARDINAL'],
 ['entity: 3, type: CARDINAL'],
 ['entity: 1, type: CARDINAL',
  'entity: 2, type: CARDINAL',
  'entity: 3, type: CARDINAL',
  'entity: 4, type: CARDINAL',
  'entity: 5, type: CARDINAL',
  'entity: 4, type: CA

## Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

In [75]:
def cbow_split(tokens: list,  ne: str, window=1, pad_token = 'PAD'):
  splits = []
  for x in range(window):
    tokens.append(pad_token)
    tokens.insert(0,pad_token)
  for i, el in enumerate(tokens):
    if el != 'PAD':
      left = tokens[i-window:i+window+1][:window]
      right = tokens[i-window:i+window+1][window+1:]
      if len(ne.split()) == 1:
        if el==ne:
          splits.append((left,el,right))
     
  return splits


# так как нам нужны только левый и правый соседи, окно = 1
sample_text = list(small_df['normalize_reviews'])[0]
sample_ne = list(small_df['final_entities'])[0]
for ne in sample_ne:
  print(cbow_split(sample_text, ne))


[(['telling'], 'product', ['shipped']), (['buy'], 'product', ['work'])]
[(['sent'], 'email', ['telling']), (['another'], 'email', ['telling']), (['received'], 'email', ['telling'])]
[(['money'], 'sent', ['email']), (['I'], 'sent', ['numerous'])]
